In [11]:
import csv
import gensim

from stop_words import get_stop_words
from nltk.stem.porter import PorterStemmer
from gensim import corpora, models

en_stop = get_stop_words('en')
p_stemmer = PorterStemmer()

# run lda on words themselves
# https://rstudio-pubs-static.s3.amazonaws.com/79360_850b2a69980c4488b1db95987a24867a.html
# whoops. didn't work
def read_docs(filepath):
    docs = []
    with open(filepath, "rb") as csvfile:
        reader = csv.reader(csvfile, delimiter=",")
        for row in reader:
            docs.append([row[0],row[1]])
    return docs

doc_set = read_docs("data/cleaned/regular_verbs_clean.csv")
doc_set += read_docs("data/cleaned/irregular_verbs_clean.csv")

num_docs = len(doc_set)
texts = []
for i in range(num_docs):
    # remove stop words from tokens
    # stem token
    texts.append([p_stemmer.stem(t) for t in doc_set[i] if not t in en_stop])

dictionary = corpora.Dictionary(texts)
# corpus is a document-term matrix.  a list of vectors equal to the number of documents. 
# In each document vector is a series of tuples
corpus = [dictionary.doc2bow(text) for text in texts]
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=10, id2word = dictionary, passes=20)
print(ldamodel.print_topics(num_topics=10, num_words=3))

[(0, u'0.014*bled + 0.014*heliograph + 0.014*exculp'), (1, u'0.011*fang + 0.011*wood + 0.011*stockpil'), (2, u'0.013*hit + 0.013*mump + 0.013*owe'), (3, u'0.019*saw + 0.013*patent + 0.013*reimagin'), (4, u'0.014*attract + 0.014*explain + 0.014*rectifi'), (5, u'0.012*need + 0.012*groan + 0.012*geld'), (6, u'0.027*let + 0.014*leav + 0.014*stultifi'), (7, u'0.014*bereav + 0.014*shut + 0.014*tangl'), (8, u'0.015*sing + 0.015*slime + 0.015*catalyz'), (9, u'0.014*dream + 0.014*bit + 0.014*bust')]


In [ ]:
import csv
import string
import sys, itertools
import random
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from mpl_toolkits.mplot3d import Axes3D
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis


N = 1000
alphabet = string.ascii_lowercase + '#' + '.'
D = len(alphabet)
z = np.ones(N)

RI_pres = np.random.rand(D, N)
RI_pres = np.where(RI_pres>0.5, 1, -1)

RI_past = np.random.rand(D, N)
RI_past = np.where(RI_past>0.5, 1, -1)

def ngram_encode(ngram_str, letter_vecs, window=3):
    vec = np.zeros(letter_vecs.shape[1])
    full_str = '#' + ngram_str + '.'
    for il, l in enumerate(full_str[:-(window-1)]):
        trivec = letter_vecs[alphabet.find(full_str[il]), :]
        for c3 in range(1, window):
            trivec = trivec * np.roll(letter_vecs[alphabet.find(full_str[il+c3]), :], c3)
        vec += trivec
    return vec

def ngram_encode_cl(ngram_str, letter_vecs, window=3):
    vec = np.zeros(letter_vecs.shape[1])
    full_str = '#' + ngram_str + '.'
    for il, l in enumerate(full_str[:-(window-1)]):
        trivec = letter_vecs[alphabet.find(full_str[il]), :]
        for c3 in range(1, window):
            trivec = trivec * np.roll(letter_vecs[alphabet.find(full_str[il+c3]), :], c3)
        vec += trivec
    return 2* (vec + 0.1*(np.random.rand(letter_vecs.shape[1])-0.5) < 0) - 1

def read_csv(filepath):
    # lists of lists where each sublist corresponds to rule
    pres, past = {}, {}
    key = 0
    num_words = 0
    with open(filepath, "rb") as csvfile:
        reader = csv.reader(csvfile, delimiter=",")
        sub_pres, sub_past = [], []
        for row in reader:
            if "#" in row[0]:
                pres[key], past[key] = sub_pres, sub_past
                sub_pres, sub_past = [], []
                key += 1
            else:
                sub_pres.append(ngram_encode_cl(row[0], RI_pres, 3))
                sub_past.append(ngram_encode_cl(row[1], RI_past, 3))
                num_words += 2
                
    X = np.zeros((num_words-4, N))
    y = np.zeros(num_words-4)
    num_classes = len(pres.keys())

    j = 0
    for i in range(num_classes):
        for pres_vec in pres[i]:
            X[j] = pres_vec
            y[j] = i
            j += 1
        for past_vec in past[i]:
            X[j] = past_vec
            y[j] = i
            j += 1
        
        #print i
    return X, y, num_classes

def plot_proj_2D(X, y, num_classes):
    d = 2
    U, s, V = np.linalg.svd(X, full_matrices=True)
    print (U.shape, s.shape, V.shape) 

    Ud = U[:, :d]
    Sd = s[:d]*np.eye(d)
    Vd = V[:d, :]
    SdVd = np.dot(Sd, Vd)
    print Ud.shape, Sd.shape, Vd.shape, SdVd.shape
    reduced_X = Ud
        
    colors = cm.rainbow(np.linspace(0, 1, num_classes))
    print colors
    plt.figure()
    plt.scatter(reduced_X[:,0], reduced_X[:,1], color = colors[y.astype(int)])
    plt.show()
    """
    

    colors = cm.rainbow(np.linspace(0, 1, num_classes))

    fig = plt.figure()
    for i, row in data.iterrows():
        plt.scatter(proj[0], proj[1], color = 
                    plotlabels[row[self.labelcol]])
    plt.show()
    """
    
"""
http://goelhardik.github.io/2016/10/04/fishers-lda/

https://pythonhosted.org/lda/api.html
http://stackoverflow.com/questions/38044811/1-d-lda-output-dimension-in-numpy
http://scikit-learn.org/stable/modules/generated/sklearn.discriminant_analysis.LinearDiscriminantAnalysis.html

irregular verbs are in own classes separated by first letter
regular verbs are in own classes separated by rules: #d, #ed, #**ed, #, #?

"""

# we're combining both past and present tense for now?
regular_X, regular_y, regular_num_classes = read_csv("data/cleaned/regular_verbs_clean_categorize.csv")
irregular_X, irregular_y, irregular_num_classes = read_csv("data/cleaned/irregular_verbs_clean_categorize.csv")
print (regular_X.shape, regular_y.shape)
regular_clf = LinearDiscriminantAnalysis(n_components=3, store_covariance=True)
regular_clf.fit(regular_X, regular_y)
print(regular_clf.predict(np.random.random(N)))
test_indices = np.random.choice(regular_X.shape[0], 20, replace=False)
regular_y_pred = regular_clf.predict(regular_X[test_indices])
plot_proj_2D(regular_X[test_indices], regular_y_pred, regular_num_classes)

((592, 1000), (592,))
[ 0.]